In [22]:
import pandas as pd
from itertools import product

In [4]:
df = pd.read_csv("data/cars-pivot.csv")
df.head()

,STM,Month,Day,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,SUMA
0,2,1,Weekend,5.516129,4.258065,2.935484,2.580645,1.516129,5.064516,6.354839,...,46.548387,41.870968,45.064516,40.096774,38.258065,27.000000,16.290323,12.225806,5.677419,SUMA1
1,2,1,Workday,2.112903,1.096774,0.725806,0.532258,0.532258,7.370968,11.580645,...,73.000000,61.790323,68.725806,54.209677,41.451613,27.741935,16.693548,15.225806,5.919355,SUMA1
2,2,2,Weekend,5.807692,4.346154,3.500000,2.923077,2.076923,4.923077,5.961538,...,45.615385,38.846154,41.615385,39.884615,33.000000,22.115385,15.846154,11.461538,6.269231,SUMA1
3,2,2,Workday,2.372881,1.627119,0.728814,0.661017,0.661017,7.322034,11.542373,...,69.677966,62.186441,68.033898,55.135593,41.338983,29.576271,17.610169,14.830508,6.101695,SUMA1
4,2,3,Weekend,7.307692,4.689655,3.217391,2.807692,1.692308,6.038462,7.846154,...,55.423077,45.384615,43.461538,40.423077,40.192308,28.500000,18.576923,11.538462,7.307692,SUMA1


In [31]:
def sum_of_mean(df, stm, day, direction):
    cols = [e for e in list(df.columns) if e not in ['STM', 'Month', 'Day', 'SUMA']]
    tmp = pd.Series(index=cols, dtype='float64')
    for col in cols:
        # get the mean of 4 (base) lowest values for each hour
        mean = df[col].sort_values()[:4].mean()
        # compute a sum of absolute values per hour
        # absolute, because you want to consider interesting negatives
        # sum is the final score (interestingness of the hour)
        hour_weight = sum(df[col])/sum(df.sum(axis=1)) if sum(df.sum(axis=1)) else 0
        # weight by relative hour traffic (to discount small increases)
        tmp[col] = ((df[col] - mean) / mean).abs().sum() * hour_weight if \
                    mean else 0
    tmp['Max'] = max(tmp)
    tmp['STM'] = stm
    tmp['Day'] = day
    tmp['SUMA'] = direction
    print(tmp)
    # return computed Series
    return tmp

In [32]:
som = pd.DataFrame()
for stm, day, direction in list(product(set(df['STM'].values), 
                                            set(df['Day'].values), 
                                            set(df['SUMA'].values))):
    subset = df[(df['STM'] == stm) & (df['Day'] == day) & (df['SUMA'] == direction)]
    som = som.append(sum_of_mean(subset, stm, day, direction), ignore_index=True)

0        0.0388251
1        0.0175023
2       0.00732763
3       0.00912569
4        0.0445554
5        0.0766154
6         0.104534
7         0.102169
8         0.103423
9         0.246663
10        0.388329
11        0.556202
12        0.503863
13        0.345846
14        0.310838
15        0.268058
16         0.29512
17        0.293527
18        0.361127
19         0.43969
20        0.390467
21        0.262662
22        0.167055
23       0.0780236
Max       0.556202
STM              2
Day        Workday
SUMA         SUMA2
dtype: object
0       0.0392274
1       0.0213776
2       0.0161898
3       0.0140455
4       0.0476269
5       0.0444548
6        0.126155
7        0.229208
8        0.249934
9        0.369736
10       0.424638
11        0.42431
12       0.374371
13       0.343108
14       0.346976
15       0.330435
16       0.400577
17       0.375395
18       0.359368
19        0.32963
20       0.289508
21       0.202543
22       0.105218
23       0.076792
Max      0.424638
STM 

0       0.0737405
1       0.0453019
2       0.0314099
3       0.0239429
4       0.0292736
5       0.0542292
6        0.100786
7        0.178452
8         0.26575
9        0.316763
10       0.347979
11       0.380967
12       0.360083
13        0.30794
14       0.241421
15       0.217599
16       0.242703
17       0.361109
18       0.650344
19       0.830071
20       0.898361
21       0.757319
22       0.480274
23       0.196663
Max      0.898361
STM            10
Day       Weekend
SUMA        SUMA2
dtype: object
0        0.150621
1       0.0782431
2       0.0485837
3       0.0316356
4       0.0383033
5        0.113152
6        0.312436
7        0.516926
8        0.689197
9        0.662421
10       0.577968
11        0.48048
12       0.383273
13       0.396726
14       0.356706
15       0.314594
16       0.237489
17        0.22449
18       0.249131
19       0.267589
20       0.266432
21       0.200256
22       0.160874
23       0.142498
Max      0.689197
STM            10
Day       Week

KeyboardInterrupt: 

In [33]:
som

,0,1,10,11,12,13,14,15,16,17,...,4,5,6,7,8,9,Day,Max,STM,SUMA
0,0.038825,0.017502,0.388329,0.556202,0.503863,0.345846,0.310838,0.268058,0.295120,0.293527,...,0.044555,0.076615,0.104534,0.102169,0.103423,0.246663,Workday,0.556202,2.0,SUMA2
1,0.039227,0.021378,0.424638,0.424310,0.374371,0.343108,0.346976,0.330435,0.400577,0.375395,...,0.047627,0.044455,0.126155,0.229208,0.249934,0.369736,Workday,0.424638,2.0,SUMA1
2,0.043465,0.027521,0.807605,1.280069,1.333151,1.061171,0.918033,0.869636,0.934704,1.083943,...,0.039535,0.060755,0.100346,0.137405,0.186571,0.396730,Weekend,2.183720,2.0,SUMA2
3,0.057631,0.041752,1.242711,1.166738,0.944232,0.867799,0.716999,0.652825,0.552221,0.358932,...,0.124490,0.274886,1.154320,1.405406,1.467314,1.376973,Weekend,1.467314,2.0,SUMA1
4,0.022478,0.009923,0.051196,0.047486,0.058618,0.057636,0.066077,0.075541,0.060985,0.073315,...,0.008556,0.034715,0.050860,0.040396,0.054217,0.052869,Workday,0.075541,3.0,SUMA2
5,0.010469,0.006207,0.074088,0.063132,0.048954,0.054618,0.051995,0.069670,0.061789,0.068868,...,0.015896,0.051357,0.117252,0.092203,0.052586,0.075657,Workday,0.117252,3.0,SUMA1
6,0.038243,0.030214,0.166681,0.165733,0.157233,0.137907,0.154906,0.142221,0.126174,0.137149,...,0.013944,0.052133,0.119291,0.145752,0.193084,0.159810,Weekend,0.197926,3.0,SUMA2
7,0.020614,0.020081,0.173712,0.152155,0.113505,0.097168,0.108642,0.108778,0.109178,0.098381,...,0.065937,0.119084,0.163112,0.166120,0.203082,0.204647,Weekend,0.204647,3.0,SUMA1
8,0.007552,0.004025,0.032007,0.035573,0.052457,0.068180,0.064452,0.086488,0.078025,0.068793,...,0.007627,0.056174,0.036911,0.060364,0.059673,0.040988,Workday,0.086488,6.0,SUMA2
9,0.018485,0.008395,0.035406,0.034417,0.048811,0.063171,0.079079,0.038987,0.034035,0.077735,...,0.002781,0.011303,0.043456,0.047282,0.036517,0.045790,Workday,0.087695,6.0,SUMA1


In [2]:
def coefficient_of_variation(df, base):
    cols = list(df.columns)
    tmp = pd.Series(index=cols)
    # reset index, otherwise indices are months
    df = df.reset_index(drop=True)
    for col in cols:
        # get the mean of 4 (base) lowest values for each hour
        # mean = df[col].sort_values()[:base].mean()
        mean = df[col].mean()
        # std of all columns? just top 4?
        std = df[col].std()
        tmp[col] = 100 * std / mean if mean else 0
    tmp['Max'] = max(tmp)
    # return computed Series
    return tmp

In [3]:
def zscore_total(df, base):
    cols = list(df.columns)
    # reset index, otherwise indices are months
    df = df.reset_index(drop=True)
    for col in cols:
        # get the mean of 4 (base) lowest values for each hour
        mean = df[col].mean()
        # compute zscore with the selected baseline
        df[col] = (df[col] - mean) / df[col].std(ddof=0)
    idx = df.max(axis=1).idxmax()
    if not pd.isna(idx):
        out = df.iloc[idx]
        out['Max'] = out.max()
        out['Month'] = idx + 1
        out['Total'] = df.sum(axis=1).max()
    else:
        out = df.iloc[0]
        out['Max'] = 0
        out['Month'] = pd.NA
        out['Total'] = 0
    # return transformed Series
    return out